# Module 06: Data Modification - INSERT, UPDATE, DELETE

**Estimated Time:** 45 minutes

## Learning Objectives

By the end of this module, you will be able to:
- Insert single and multiple rows with INSERT
- Update existing records with UPDATE
- Delete records safely with DELETE
- Understand transaction basics (BEGIN, COMMIT, ROLLBACK)
- Follow best practices for data modification
- Avoid common pitfalls when modifying data

**WARNING:** This module modifies data. We'll create a test database to practice safely.

In [ ]:
# Setup
import sqlite3
import pandas as pd
from pathlib import Path
import shutil

%load_ext sql

# Create a copy of the database for safe testing
DB_PATH = Path.cwd().parent / "data" / "databases" / "ecommerce.db"
TEST_DB_PATH = Path.cwd().parent / "data" / "databases" / "ecommerce_test.db"

# Copy the database
if TEST_DB_PATH.exists():
    TEST_DB_PATH.unlink()
shutil.copy(DB_PATH, TEST_DB_PATH)

# Connect to test database
conn = sqlite3.connect(TEST_DB_PATH)
%sql sqlite:///$TEST_DB_PATH

print("✓ Connected to test database (ecommerce_test.db)")
print("  All modifications will be made to the test copy only.")

## 1. INSERT: Adding New Data

The INSERT statement adds new rows to a table.

### Syntax
```sql
-- Insert single row
INSERT INTO table_name (column1, column2, ...)
VALUES (value1, value2, ...);

-- Insert multiple rows
INSERT INTO table_name (column1, column2, ...)
VALUES 
    (value1a, value2a, ...),
    (value1b, value2b, ...),
    (value1c, value2c, ...);
```

In [ ]:
# First, let's see the current categories
%%sql
SELECT * FROM categories ORDER BY category_id

In [ ]:
# Insert a single category
%%sql
INSERT INTO categories (category_name, description)
VALUES ('Outdoor Gear', 'Equipment for outdoor activities and camping');

In [ ]:
# Verify the insert
%%sql
SELECT * FROM categories ORDER BY category_id DESC LIMIT 3

In [ ]:
# Insert multiple rows at once
%%sql
INSERT INTO categories (category_name, description)
VALUES 
    ('Pet Supplies', 'Products for pets and animal care'),
    ('Garden & Tools', 'Gardening equipment and tools'),
    ('Automotive', 'Car parts and accessories');

In [ ]:
# Verify multiple inserts
%%sql
SELECT * FROM categories ORDER BY category_id DESC LIMIT 5

In [ ]:
# Insert with specific column values (other columns get defaults)
cursor = conn.cursor()
cursor.execute(
    """
    INSERT INTO products (product_name, category_id, price, stock_quantity)
    VALUES ('Test Product', 1, 99.99, 50)
"""
)
conn.commit()

print(f"✓ Inserted product with ID: {cursor.lastrowid}")

In [ ]:
# Verify the product insert
%%sql
SELECT * FROM products ORDER BY product_id DESC LIMIT 3

## 2. INSERT with SELECT: Bulk Insert

You can insert data from a SELECT query.

In [ ]:
# Create a temporary table for demo
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS high_value_customers (
        customer_id INTEGER,
        customer_name TEXT,
        total_spent REAL,
        order_count INTEGER
    )
"""
)
conn.commit()
print("✓ Created high_value_customers table")

In [ ]:
# Insert data from a SELECT query
cursor.execute(
    """
    INSERT INTO high_value_customers (customer_id, customer_name, total_spent, order_count)
    SELECT 
        c.customer_id,
        c.first_name || ' ' || c.last_name,
        SUM(o.total_amount),
        COUNT(o.order_id)
    FROM customers c
    INNER JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
    HAVING SUM(o.total_amount) > 300
"""
)
conn.commit()
print(f"✓ Inserted {cursor.rowcount} high-value customers")

In [ ]:
# View the results
%%sql
SELECT * FROM high_value_customers ORDER BY total_spent DESC LIMIT 10

## 3. UPDATE: Modifying Existing Data

The UPDATE statement modifies existing rows.

### Syntax
```sql
UPDATE table_name
SET column1 = value1, column2 = value2, ...
WHERE condition;
```

**CRITICAL:** Always use WHERE with UPDATE unless you want to update ALL rows!

In [ ]:
# Check product before update
%%sql
SELECT product_id, product_name, price, stock_quantity
FROM products
WHERE product_name = 'Test Product'

In [ ]:
# Update a single row
cursor.execute(
    """
    UPDATE products
    SET price = 79.99, stock_quantity = 100
    WHERE product_name = 'Test Product'
"""
)
conn.commit()
print(f"✓ Updated {cursor.rowcount} row(s)")

In [ ]:
# Verify the update
%%sql
SELECT product_id, product_name, price, stock_quantity
FROM products
WHERE product_name = 'Test Product'

In [ ]:
# Update multiple rows with WHERE clause
cursor.execute(
    """
    UPDATE products
    SET price = price * 1.10
    WHERE category_id = 1 AND price < 100
"""
)
conn.commit()
print(f"✓ Applied 10% price increase to {cursor.rowcount} product(s)")

In [ ]:
# Update based on calculated values
cursor.execute(
    """
    UPDATE products
    SET stock_quantity = stock_quantity + 50
    WHERE stock_quantity < 30
"""
)
conn.commit()
print(f"✓ Restocked {cursor.rowcount} low-inventory product(s)")

## 4. DELETE: Removing Data

The DELETE statement removes rows from a table.

### Syntax
```sql
DELETE FROM table_name
WHERE condition;
```

**CRITICAL:** Always use WHERE with DELETE unless you want to delete ALL rows!

In [ ]:
# Count rows before delete
%%sql
SELECT COUNT(*) as count FROM products WHERE product_name = 'Test Product'

In [ ]:
# Delete a specific row
cursor.execute(
    """
    DELETE FROM products
    WHERE product_name = 'Test Product'
"""
)
conn.commit()
print(f"✓ Deleted {cursor.rowcount} row(s)")

In [ ]:
# Verify deletion
%%sql
SELECT COUNT(*) as count FROM products WHERE product_name = 'Test Product'

In [ ]:
# Delete multiple rows
cursor.execute(
    """
    DELETE FROM categories
    WHERE category_id > 10
"""
)
conn.commit()
print(f"✓ Deleted {cursor.rowcount} test categories")

## 5. Transactions: ACID Properties

Transactions ensure data integrity by grouping multiple operations.

### ACID Properties:
- **Atomicity**: All or nothing - either all operations succeed or all fail
- **Consistency**: Database remains in a valid state
- **Isolation**: Concurrent transactions don't interfere
- **Durability**: Committed changes are permanent

### Commands:
- **BEGIN**: Start a transaction
- **COMMIT**: Save all changes
- **ROLLBACK**: Undo all changes since BEGIN

In [ ]:
# Example: Transaction with COMMIT
cursor.execute("BEGIN TRANSACTION")

try:
    # Insert a new customer
    cursor.execute(
        """
        INSERT INTO customers (first_name, last_name, email, city, country)
        VALUES ('John', 'Doe', 'john.doe@example.com', 'New York', 'USA')
    """
    )
    customer_id = cursor.lastrowid

    # Insert an order for that customer
    cursor.execute(
        """
        INSERT INTO orders (customer_id, order_date, status, total_amount)
        VALUES (?, date('now'), 'Pending', 150.00)
    """,
        (customer_id,),
    )

    # If everything succeeded, commit
    conn.commit()
    print("✓ Transaction committed successfully")
    print(f"  Customer ID: {customer_id}")

except Exception as e:
    conn.rollback()
    print(f"✗ Transaction failed: {e}")
    print("  All changes rolled back")

In [ ]:
# Example: Transaction with ROLLBACK
cursor.execute("BEGIN TRANSACTION")

# Make some changes
cursor.execute(
    """
    UPDATE products
    SET price = price * 2
    WHERE category_id = 1
"""
)

print(f"Changed {cursor.rowcount} prices (not committed yet)")

# Decide to rollback instead
conn.rollback()
print("✓ Transaction rolled back - no changes saved")

In [ ]:
# Verify rollback worked (prices should be unchanged)
%%sql
SELECT product_name, price 
FROM products 
WHERE category_id = 1 
ORDER BY price DESC 
LIMIT 5

## 6. Best Practices for Data Modification

### DO:
1. **Always use WHERE** with UPDATE and DELETE
2. **Test with SELECT first** - convert your WHERE clause to a SELECT to verify
3. **Use transactions** for multiple related operations
4. **Backup data** before bulk modifications
5. **Use proper data types** when inserting
6. **Validate input** to prevent SQL injection

### DON'T:
1. **Don't UPDATE/DELETE without WHERE** (unless intentional)
2. **Don't ignore foreign key constraints**
3. **Don't insert duplicate data** unnecessarily
4. **Don't use string concatenation** for values (use parameterized queries)

In [ ]:
# BEST PRACTICE: Test DELETE with SELECT first

# Step 1: Use SELECT to see what would be deleted
%%sql
SELECT * FROM high_value_customers WHERE total_spent < 350

In [ ]:
# Step 2: If the SELECT looks correct, convert to DELETE
cursor.execute(
    """
    DELETE FROM high_value_customers 
    WHERE total_spent < 350
"""
)
conn.commit()
print(f"✓ Deleted {cursor.rowcount} row(s)")

## 7. Real-World Examples

In [ ]:
# Example 1: Bulk price update for a sale
cursor.execute("BEGIN TRANSACTION")

try:
    # 20% off all products in category 2
    cursor.execute(
        """
        UPDATE products
        SET price = price * 0.80
        WHERE category_id = 2
    """
    )

    affected = cursor.rowcount
    conn.commit()
    print(f"✓ Applied 20% discount to {affected} products")

except Exception as e:
    conn.rollback()
    print(f"✗ Sale update failed: {e}")

In [ ]:
# Example 2: Update order status
cursor.execute(
    """
    UPDATE orders
    SET status = 'Shipped'
    WHERE status = 'Pending' 
      AND order_date < date('now', '-7 days')
"""
)
conn.commit()
print(f"✓ Updated {cursor.rowcount} pending orders to shipped")

## 8. Exercises

Practice what you've learned with these exercises.

### Exercise 1: Insert New Category and Products
Insert a new category called 'Office Supplies' and then insert 3 products in that category.

In [ ]:
# Your code here

### Exercise 2: Update Product Prices
Increase the price by 5% for all products with stock_quantity less than 20.

In [ ]:
# Your code here

### Exercise 3: Create and Populate Summary Table
Create a table called `monthly_sales` and populate it with monthly sales totals using INSERT...SELECT.

In [ ]:
# Your code here

### Exercise 4: Transaction Practice
Using a transaction, insert a new customer and an order for that customer. If the order total is invalid (negative), rollback the entire transaction.

In [ ]:
# Your code here

### Exercise 5: Safe DELETE
First use SELECT to preview, then delete all orders with status 'Cancelled' and order_date older than 1 year.

In [ ]:
# Your code here

## Summary

In this module, you learned:
- ✓ Inserting single and multiple rows with INSERT
- ✓ Bulk insert using INSERT...SELECT
- ✓ Updating existing records with UPDATE
- ✓ Safely deleting records with DELETE
- ✓ Using transactions (BEGIN, COMMIT, ROLLBACK)
- ✓ Best practices for data modification

**Key Takeaways:**
- Always use WHERE with UPDATE and DELETE
- Test with SELECT before UPDATE/DELETE
- Use transactions for multiple related operations
- COMMIT saves changes, ROLLBACK undoes them
- Parameterized queries prevent SQL injection
- Backup data before bulk modifications

**Next:** Module 07 - Database Design

In [ ]:
# Cleanup
conn.close()

# Optional: Delete test database
# TEST_DB_PATH.unlink()

print("✓ Database connection closed")
print("  Test database preserved at:", TEST_DB_PATH)